近期公司内部，经常会因为客户的紧急需求，需要拉取全部设备的信息。
在我们的配置存储格式中，会有许许多多的config，如果每次都需要自己手动去整理的话，非常的麻烦。所以我希望能够将它们的一些信息整理成表格列出来。

思路：
1.读取目录下的所有配置文件名，例 20180128 shcesw2A 与 20180129 shcesw2A；
我希望通过统一的命名格式，通过**设备名**，确认同设备，再通过**日期**来判断配置新旧。进行信息的筛选。

2.通过配置筛选出各个厂商对应的关键信息，具体如下：
设备地点，楼层，房间，设备型号，环境，主机名，厂商EOL，SN，后续型号，L3链接信息；

3.将信息筛选完毕后，逐一输出；

In [10]:
import os
import numpy as np
import textfsm
import csv
def get_path(path):
    #申明全局
    global all_path
    for root, dirs, files in os.walk(path):
        for filepath in files:
            #print(os.path.join(root, filepath))
            all_path.append(os.path.join(root, filepath))
            
if __name__ == "__main__":
    path = "O:\py" 
all_path=[]
get_path(path)
path_t_all = []
all_path_str = ','.join(all_path)
print(all_path_str)

O:\py\Device1.py,O:\py\lv7.py,O:\py\test_commands.csv,O:\py\test_devices.csv,O:\py\I-01\20160409_L21c.txt,O:\py\I-01\20160559_L21c.txt,O:\py\J-01\20180923_L33c.txt,O:\py\J-01\20181111_L33c.txt,O:\py\T-05\20160666_RT1.txt,O:\py\T-05\20199999_RT1.txt


In [11]:
def export(now):   
    #筛选所有的设备list
    global all_device
    global all_device_date
    global all_device_device
    link_start=now.find('20')
    link_end=now.find('_',link_start)
   
    name_start=now.find('_',link_start-1)
    name_end=now.find('.txt',name_start)
    
    if now[link_start:link_end].startswith("20"): 
        #print(now[link_start:link_end]+now[name_start+1:name_end])          
        print('date:---',now[link_start:link_end])
        print('hostname:---',now[name_start+1:name_end])    
        print('完整:---',now[link_start:link_end]+now[name_start:name_end])
        all_device_date.append(now[link_start:link_end])
        all_device_device.append(now[name_start+1:name_end])
        all_device.append(now[link_start:link_end]+now[name_start:name_end])
        return name_end                          
    else:
        return -1    
label=0
path_t = []
all_device = []
all_device_date = [] 
all_device_device = []
nput_text = 0
row_all = []

while True:
    all_path_str=all_path_str[label:]
    label=export(all_path_str)
    if label == -1:
            break;

date:--- 20160409
hostname:--- L21c
完整:--- 20160409_L21c
date:--- 20160559
hostname:--- L21c
完整:--- 20160559_L21c
date:--- 20180923
hostname:--- L33c
完整:--- 20180923_L33c
date:--- 20181111
hostname:--- L33c
完整:--- 20181111_L33c
date:--- 20160666
hostname:--- RT1
完整:--- 20160666_RT1
date:--- 20199999
hostname:--- RT1
完整:--- 20199999_RT1


In [12]:
#print(all_device)
#print(all_path_str)
#筛选出对应config的对应path
#all_path = all_path.split(",")
#print(all_path)
#all_path_ = all_path.replace("\\", "\")


for i in range (0,len(all_device)):    
    for j in range (0,len(all_path)):
        if all_device[i] in all_path[j]:
            path_t.append(all_path[j])
        else:
            pass
print(str(len(path_t))+str(path_t))
print(str(len(all_device_device))+str(all_device_device))
 

6['O:\\py\\I-01\\20160409_L21c.txt', 'O:\\py\\I-01\\20160559_L21c.txt', 'O:\\py\\J-01\\20180923_L33c.txt', 'O:\\py\\J-01\\20181111_L33c.txt', 'O:\\py\\T-05\\20160666_RT1.txt', 'O:\\py\\T-05\\20199999_RT1.txt']
6['L21c', 'L21c', 'L33c', 'L33c', 'RT1', 'RT1']


In [13]:
#踢出old的配置文件，暂未解决
for i in range(0,len(path_t)):
    #print("*",all_device)
    #print("*",all_device_date)
    #print("*",all_device_device)
    #print("&",path_t)
    global path_t_all
    type(path_t_all)
    
    if (len(path_t) == 0):
        pass
    while all_device_device[i] == all_device_device[i-1]:
        #print(all_device_date[i]+"\n"+all_device_date[i-1])
        if all_device_date[i] <= all_device_date[i-1]:
            #all_device_device.pop(i)
            path_t_all.append(path_t[i-1])
        else:
            #all_device_device.pop(i-1)
            #path_t.pop(i-1)
            path_t_all.append(path_t[i])
            break
    else:
        pass
print(path_t_all)

['O:\\py\\I-01\\20160559_L21c.txt', 'O:\\py\\J-01\\20181111_L33c.txt', 'O:\\py\\T-05\\20199999_RT1.txt']


In [14]:
#根据目录读取txt
print(all_device_device)
print(path_t_all)
out = open('Z:\\common\\006 Lab\\NetDevOps\\13网络设备的配置管理\\export_config.csv','w',newline='')#这种方式会覆盖之前的内容。
csv_writer = csv.writer(out)
         
        #print(row_all)
for i in range(0,len(path_t_all)):
    global row_all
    TEMP_FILE="Z:\\common\\006 Lab\\NetDevOps\\13网络设备的配置管理\\textFSM.cfg" 
    INPUT_FILE=str(path_t_all[i])
    #打开模版文件，并且初始化FSM类
    fsm = textfsm.TextFSM(open(TEMP_FILE))
    input_text = open(INPUT_FILE).read()
    #使用parseText方式解析文件
    fsm_results = fsm.ParseText(input_text)
    #print(fsm.header)
    for row in fsm_results:
        print(row)         
        csv_writer.writerow(row)
        
out.close()

['L21c', 'L21c', 'L33c', 'L33c', 'RT1', 'RT1']
['O:\\py\\I-01\\20160559_L21c.txt', 'O:\\py\\J-01\\20181111_L33c.txt', 'O:\\py\\T-05\\20199999_RT1.txt']
['L21', 'FOC1966666', 'WS-C2960+24TC-L', '15.2(1)E', 'flash:/c2960-lanbasek9-mz.152-1.E.bin']
['L33', 'FO7777777', 'WS-C2960S-24TS-L', '12.2(58)SE1', 'flash:/c2960s-universalk9-mz.122-58.SE1.bin']
['router1', 'FGL19999999', 'CISCO2901/K9', '15.1(4)M3', 'flash0:c2900-universalk9-mz.SPA.151-4.M3.bin']
